In [1]:
from sklearn.ensemble import BaggingRegressor
import xgboost as xgb
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import ktrain
from ktrain import tabular
import os
import glob
import optuna
import pickle
import math
import matlab
import matlab.engine
from qiskit_ibm_provider import IBMProvider
with open("API_KEY.txt","r") as file:
    key = file.read()
provider = IBMProvider(token=key)
def filters(x):
    if "simulator" not in x.name:
        return x
backends = provider.backends(filters=filters)
backends = sorted(backends,key=lambda x: x.name)
backends

[<IBMBackend('ibm_lagos')>,
 <IBMBackend('ibm_nairobi')>,
 <IBMBackend('ibm_perth')>,
 <IBMBackend('ibmq_belem')>,
 <IBMBackend('ibmq_jakarta')>,
 <IBMBackend('ibmq_lima')>,
 <IBMBackend('ibmq_manila')>,
 <IBMBackend('ibmq_quito')>]

In [2]:
def read_program(program_name,backend):
    all_files = glob.glob(os.path.join("./real_circuits_hardware/", "*.csv"))
    for file in all_files:
        if program_name in file and backend.name in file:
            df = pd.read_csv(file)
            cols = [x for x in df.columns if x not in ["Avg_inverted_error","observed_prob_25","observed_prob_75"]]
            return df.loc[:,cols]
        
def read_program_qraft(program_name,backend):
    all_files = glob.glob(os.path.join("./real_circuits_hardware_qraft/", "*.csv"))
    for file in all_files:
        if program_name in file and backend.name in file:
            df = pd.read_csv(file)
            return df
        
def get_program_dict():
    program = {}
    for file in os.listdir("./real_circuits"):
        name = file.split("_")[0]
        program[name] = {"name":name,"LR":0,"Lasso":0,"Ridge":0,"Elastic":0,"SVR":0,"KNN":0,
                         "lgbm":0,"xgbm":0,"edt":0,"mlp":0,"qraft":0,"noise":0}
    return program

def HellingerDistance(p, q, qraft=False):
    p = p/100
    q = q/100
    if not qraft:
        p[p<=0] = 0
        q[q<=0] = 0
        if q.sum()>1:
            q[q>0] = q[q>0]-((q.sum()-p.sum())/(len(q)-len(q[q==0])))
            q[q<0] = 0
    
    n = len(p)
    sum_ = 0.0
    for i in range(n):
        sum_ += (np.sqrt(p[i]) - np.sqrt(q[i]))**2
    result = (1.0 / np.sqrt(2.0)) * np.sqrt(sum_)
    return result

In [6]:
# load models
lgbm = lgb.Booster(model_file='lgbm.txt')
xgbm = xgb.Booster()
xgbm.load_model("xgbm.json")
with open("edt.pkl","rb") as file:
    edt = pickle.load(file)
mlp_predictor = ktrain.load_predictor("mlp")
with open("lreg.pkl","rb") as file:
    lreg = pickle.load(file)
with open("Rreg.pkl","rb") as file:
    Rreg = pickle.load(file)
with open("lareg.pkl","rb") as file:
    Lareg = pickle.load(file)
with open("Ereg.pkl","rb") as file:
    Ereg = pickle.load(file)
with open("Sreg.pkl","rb") as file:
    Sreg = pickle.load(file)
with open("Kreg.pkl","rb") as file:
    Kreg = pickle.load(file)
with open("QRAFT.pkl","rb") as file:
    QRAFT = pickle.load(file)

In [7]:
backend_dict = {}
for backend in backends:
    
    RealPrograms = get_program_dict()
    
    for program in RealPrograms.keys():
        try:
            data = read_program(RealPrograms[program]['name'],backend)
            target = data.pop("target")
            preds = lreg.predict(data.values)
            RealPrograms[program]["LR"] = np.round(HellingerDistance(target.values,preds),2)
            preds = Rreg.predict(data.values)
            RealPrograms[program]["Ridge"] = np.round(HellingerDistance(target.values,preds),2)
            preds = Lareg.predict(data.values)
            RealPrograms[program]["Lasso"] = np.round(HellingerDistance(target.values,preds),2)
            preds = Ereg.predict(data.values)
            RealPrograms[program]["Elastic"] = np.round(HellingerDistance(target.values,preds),2)
            preds = Sreg.predict(data.values)
            RealPrograms[program]["SVR"] = np.round(HellingerDistance(target.values,preds),2)
            preds = Kreg.predict(data.values)
            RealPrograms[program]["KNN"] = np.round(HellingerDistance(target.values,preds),2)
        except:
            continue
    
    for program in RealPrograms.keys():
        try:
            data = read_program(RealPrograms[program]['name'],backend)
            target = data.pop("target")
            preds = lgbm.predict(data)
            RealPrograms[program]["lgbm"] = np.round(HellingerDistance(target.values,preds),2)
            RealPrograms[program]["noise"] = np.round(HellingerDistance(target.values,data["observed_prob_50"].values,True),2)
        except:
            continue

    for program in RealPrograms.keys():
        try:
            data = read_program(RealPrograms[program]['name'],backend)
            target = data.pop("target")
            dtest = xgb.DMatrix(data,target)
            preds = xgbm.predict(dtest)
            RealPrograms[program]["xgbm"] = np.round(HellingerDistance(target.values,preds),2)
        except:
            continue

    for program in RealPrograms.keys():
        try:
            data = read_program(RealPrograms[program]['name'],backend)
            target = data.pop("target")
            preds = edt.predict(data)
            RealPrograms[program]["edt"] = np.round(HellingerDistance(target.values,preds),2)
        except:
            continue

    
    for program in RealPrograms.keys():
        try:
            data = read_program(RealPrograms[program]['name'],backend)
            preds = mlp_predictor.predict(data)
            RealPrograms[program]["mlp"] = np.round(HellingerDistance(data["target"].values,preds)[0],2)
            RealPrograms[program]["qraft"] = QRAFT[backend.name][program]["hqraft"]
        except:
            continue

    backend_dict[backend.name] = RealPrograms

['target']
   target
0    0.00
1    7.03
2    0.00
3    1.37
4    1.37
['target']
      target
0   0.200000
1  40.040001
2   0.000000
3   0.000000
4   0.000000
['target']
      target
0   0.000000
1  28.610001
2   0.000000
3   0.000000
4   0.000000
['target']
   target
0    0.00
1    0.00
2    0.00
3   10.35
4    8.59
['target']
   target
0     0.0
1     0.0
2     0.0
3   100.0
['target']
   target
0    0.98
1    0.49
2    8.89
3    0.00
4    0.29
['target']
   target
0    0.00
1    9.96
2    0.00
3    1.86
4    1.17
['target']
      target
0   0.200000
1  37.209999
2   0.000000
3   0.000000
4   0.000000
['target']
      target
0   0.000000
1  28.610001
2   0.000000
3   0.000000
4   0.000000
['target']
   target
0    0.00
1    0.00
2    0.00
3   10.35
4   10.74
['target']
   target
0     0.0
1     0.0
2     0.0
3   100.0
['target']
   target
0    0.98
1    0.49
2    8.11
3    0.00
4    0.49
['target']
   target
0    0.00
1    7.03
2    0.00
3    1.37
4    1.37
['target']
      target
0

In [8]:
from IPython.display import display
reform = {(outerKey, innerKey): values for outerKey, innerDict in backend_dict.items() for innerKey, values in innerDict.items()}
df = pd.DataFrame.from_dict(reform,orient='index')
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(df)

name    LR  Lasso  Ridge  Elastic   SVR   
ibm_lagos    groundstate  groundstate  0.57   0.58   0.57     0.58  0.54  \
             pricingcall  pricingcall  0.66   0.63   0.66     0.64  0.65   
             pricingput    pricingput  0.64   0.60   0.64     0.61  0.67   
             qaoa                qaoa  0.61   0.62   0.61     0.62  0.59   
             routing          routing  0.00   0.04   0.00     0.04  0.31   
             tsp                  tsp  0.42   0.43   0.42     0.43  0.47   
ibm_nairobi  groundstate  groundstate  0.52   0.53   0.52     0.53  0.51   
             pricingcall  pricingcall  0.62   0.59   0.62     0.61  0.61   
             pricingput    pricingput  0.64   0.59   0.64     0.61  0.68   
             qaoa                qaoa  0.68   0.68   0.68     0.68  0.60   
             routing          routing  0.03   0.03   0.03     0.03  0.34   
             tsp                  tsp  0.55   0.55   0.55     0.55  0.51   
ibm_perth    groundstate  groundstate  0.50   0.50   0.50     0.50  0.50   
             pricingcall  pricingcall  0.60   0.57   0.60     0.59  0.62   
             pricingput    pricingput  0.67   0.64   0.67     0.65  0.68   
             qaoa                qaoa  0.64   0.65   0.64     0.65  0.60   
             routing          routing  0.09   0.11   0.09     0.11  0.35   
             tsp                  tsp  0.49   0.49   0.49     0.49  0.50   
ibmq_belem   groundstate  groundstate  0.67   0.67   0.67     0.67  0.59   
             pricingcall  pricingcall  0.83   0.80   0.83     0.81  0.71   
             pricingput    pricingput  0.84   0.81   0.84     0.82  0.71   
             qaoa                qaoa  0.62   0.63   0.62     0.62  0.59   
             routing          routing  0.57   0.58   0.57     0.58  0.55   
             tsp                  tsp  0.57   0.57   0.57     0.57  0.53   
ibmq_jakarta groundstate  groundstate  0.55   0.56   0.55     0.56  0.53   
             pricingcall  pricingcall  0.54   0.49   0.54     0.52  0.58   
             pricingput    pricingput  0.57   0.54   0.57     0.56  0.62   
             qaoa                qaoa  0.62   0.63   0.62     0.63  0.59   
             routing          routing  0.04   0.03   0.04     0.04  0.33   
             tsp                  tsp  0.34   0.35   0.34     0.35  0.44   
ibmq_lima    groundstate  groundstate  0.34   0.36   0.34     0.35  0.44   
             pricingcall  pricingcall  0.79   0.76   0.79     0.77  0.70   
             pricingput    pricingput  0.82   0.80   0.82     0.81  0.71   
             qaoa                qaoa  0.61   0.62   0.61     0.61  0.59   
             routing          routing  0.03   0.03   0.03     0.03  0.34   
             tsp                  tsp  0.30   0.30   0.30     0.30  0.42   
ibmq_manila  groundstate  groundstate  0.52   0.53   0.52     0.53  0.51   
             pricingcall  pricingcall  0.61   0.60   0.61     0.61  0.62   
             pricingput    pricingput  0.63   0.61   0.63     0.62  0.71   
             qaoa                qaoa  0.46   0.50   0.46     0.49  0.54   
             routing          routing  0.03   0.02   0.03     0.02  0.34   
             tsp                  tsp  0.13   0.14   0.13     0.14  0.37   
ibmq_quito   groundstate  groundstate  0.36   0.37   0.36     0.37  0.44   
             pricingcall  pricingcall  0.76   0.73   0.76     0.74  0.71   
             pricingput    pricingput  0.61   0.58   0.61     0.58  0.66   
             qaoa                qaoa  0.40   0.44   0.40     0.43  0.52   
             routing          routing  0.07   0.10   0.07     0.09  0.36   
             tsp                  tsp  0.28   0.29   0.28     0.29  0.42   

                           KNN  lgbm  xgbm   edt   mlp  qraft  noise  
ibm_lagos    groundstate  0.56  0.55  0.56  0.54  0.50   0.62   0.59  
             pricingcall  0.59  0.65  0.63  0.61  0.57   0.59   0.71  
             pricingput   0.79  0.60  0.63  0.56  0.54   0.66   0.69  
             qaoa       

In [9]:
new_df = df.reset_index()
new_df = new_df[new_df["noise"]!=0]
programs = {}

for program_name in np.unique(new_df["level_1"].values):
    errors = {}
    for col in new_df.columns:
        if "name" not in col and "level" not in col:
            errors[col] = np.round(new_df[new_df["level_1"]==program_name][col].mean(),2)
    
    programs[program_name] = errors

In [10]:
avg_df = pd.DataFrame.from_dict(programs,orient='index')
avg_df.to_csv("outputerror_application_level_hardware.csv")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(avg_df)

,LR,Lasso,Ridge,Elastic,SVR,KNN,lgbm,xgbm,edt,mlp,qraft,noise
groundstate,0.50,0.51,0.50,0.51,0.51,0.52,0.47,0.47,0.47,0.45,0.60,0.55
pricingcall,0.68,0.65,0.68,0.66,0.65,0.68,0.67,0.66,0.62,0.60,0.62,0.72
pricingput,0.68,0.65,0.68,0.66,0.68,0.66,0.69,0.68,0.59,0.59,0.63,0.72
qaoa,0.58,0.60,0.58,0.59,0.58,0.64,0.59,0.63,0.59,0.57,0.48,0.64
routing,0.11,0.12,0.11,0.12,0.36,0.14,0.24,0.23,0.25,0.10,0.72,0.28
tsp,0.38,0.39,0.38,0.39,0.46,0.43,0.37,0.38,0.36,0.36,0.66,0.43


In [11]:
new_df_bk = df.reset_index()
new_df_bk = new_df_bk[new_df_bk["noise"]!=0]
computers = {}

for computer_name in np.unique(new_df["level_0"].values):
    errors = {}
    for col in new_df.columns:
        if "name" not in col and "level" not in col:
            errors[col] = np.round(new_df_bk[new_df_bk["level_0"]==computer_name][col].mean(),2)
    
    computers[computer_name] = errors

In [12]:
avg_df_bk = pd.DataFrame.from_dict(computers,orient='index')
avg_df_bk.to_csv("outputerror_computer_level_hardware.csv")
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(avg_df_bk)

,LR,Lasso,Ridge,Elastic,SVR,KNN,lgbm,xgbm,edt,mlp,qraft,noise
ibm_lagos,0.48,0.48,0.48,0.49,0.54,0.51,0.49,0.50,0.47,0.44,0.60,0.54
ibm_nairobi,0.51,0.49,0.51,0.50,0.54,0.53,0.49,0.52,0.50,0.46,0.61,0.56
ibm_perth,0.50,0.49,0.50,0.50,0.54,0.49,0.49,0.51,0.48,0.47,0.64,0.56
ibmq_belem,0.68,0.68,0.68,0.68,0.61,0.65,0.70,0.67,0.64,0.65,0.61,0.69
ibmq_jakarta,0.44,0.43,0.44,0.44,0.52,0.49,0.44,0.44,0.43,0.38,0.61,0.52
ibmq_lima,0.48,0.48,0.48,0.48,0.53,0.48,0.50,0.50,0.47,0.44,0.63,0.55
ibmq_manila,0.40,0.40,0.40,0.40,0.52,0.50,0.48,0.48,0.41,0.35,0.63,0.50
ibmq_quito,0.41,0.42,0.41,0.42,0.52,0.43,0.45,0.44,0.42,0.36,0.62,0.52
